In [1]:
import numpy as np

In [2]:
def mod_housholder(A):
    u = np.zeros([A.shape[0],A.shape[1]])    
    I = np.eye(A.shape[0])
    Q = I
    for k in range(A.shape[1]):
        print(k)
        Amax = np.max(A[:,k])
        alpha = 0
        A[k,k] = -np.sign(A[k,k])*Amax*alpha
        for i in range(A.shape[0]):
            u[i,k] = A[i,k]/Amax
            alpha = alpha + np.abs(u[i,k])**2
        alpha = np.sqrt(alpha)
        betha = 1/(alpha*(alpha+np.abs(u[k,k])))
        u[k,k] = u[k,k] + np.sign(A[k,k])*alpha
        Q_vor = I-2*np.outer(u[:,k],u[:,k])/(np.vdot(u[:,k],u[:,k]))
        Q = Q.dot(Q_vor)
        #A[k,k] = -np.sign(A[k,k])*Amax*alpha
        #for i in range(k+1,A.shape[0]):
        #    A[i,k] = 0
        #for j in range(k+1,A.shape[1]):
        #    s = betha*np.vdot(u[:,k],A[:,j])
        #    for i in range(k,A.shape[0]):
        #        A[i,j] = A[i,j] - u[i,k]*s
           
    return Q,A

In [ ]:
from netgen import gui
from ngsolve import *
from netgen.geom2d import SplineGeometry
import scipy.sparse as sp
import numpy as np
from math import pi

In [ ]:
from netgen.geom2d import SplineGeometry
geo = SplineGeometry()
geo.AddRectangle( (0, 0), (2, 0.41), bcs = ("wall", "outlet", "wall", "inlet"))
geo.AddCircle ( (0.2, 0.2), r=0.05, leftdomain=0, rightdomain=1, bc="cyl")
mesh = Mesh( geo.GenerateMesh(maxh=0.08))
mesh.Curve(3)
Draw(mesh)
# viscosity
nu = 0.001
dt = 0.001
dn = 1./dt
z = 100  # Number of snapshots

In [ ]:
k = 3
V = VectorH1(mesh,order=k, dirichlet="wall|cyl|inlet")
Q = H1(mesh,order=k-1)
X = FESpace([V,Q])

In [ ]:
gfu_h = GridFunction(X)
gfu_N = GridFunction(X)
gfu = GridFunction(X)
velocity = gfu.components[0]


uin = CoefficientFunction((1.5*4*y*(0.41-y)/(0.41*0.41),0))
gfu_h.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))
gfu.components[0].Set(uin, definedon=mesh.Boundaries("inlet"))

Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)
Draw(gfu.components[0],mesh,"gfu",sd=3)


In [ ]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu_h.vec
gfu_h.vec.data += inv_stokes * res
gfu_N = gfu_h
Draw(gfu_h.components[0],mesh,"gfu_h",sd=3)

In [ ]:
(u,p), (v,q) = X.TnT()

a = BilinearForm(X)
stokes = (nu*InnerProduct(grad(u),grad(v))-div(u)*q-div(v)*p)*dx
a += stokes
a.Assemble()

f = LinearForm(X)
f.Assemble()

inv_stokes = a.mat.Inverse(X.FreeDofs())

res = f.vec.CreateVector()
res.data = f.vec - a.mat*gfu.vec
gfu.vec.data += inv_stokes * res
Draw(gfu.components[0],mesh,"gfu",sd=3)

In [ ]:
# matrix for implicit part of IMEX(1) scheme:
mstar = BilinearForm(X)

mstar += InnerProduct(u,v)*dx + dt*stokes

mstar.Assemble()

mh = BilinearForm(X)

mh += InnerProduct(u,v)*dx
mh.Assemble()

inv = mstar.mat.Inverse(X.FreeDofs())


conv = LinearForm(X)
conv += InnerProduct(grad(velocity)*velocity,v)*dx

In [ ]:
rows,cols,vals = a.mat.COO()
Ah = sp.csr_matrix((vals,(rows,cols)))
Fh = f.vec
rows,cols,vals = mh.mat.COO()
Mh = sp.csr_matrix((vals,(rows,cols)))

In [ ]:
t = 0
tend = 0
erg_h = np.zeros([len(gfu_h.vec),int(dn)+1])
conv_h = np.zeros([len(gfu_h.vec),int(dn)])
V = np.zeros([len(gfu.vec),z]) # creat the orthogonal transformationsmatrix for the reducebasis method
line = np.linspace(0,dn-1,z) # stors the number of linear distributed number of iteration from the solution
S_POD = np.zeros([len(gfu.vec),z])
#line = np.linspace(0,z,z) # stors the number of linear distributed number of iteration from the solution

In [ ]:
# implicit Euler/explicit Euler splitting method:
V_sb = MultiVector(gfu.vec, 1)
V_sb[0] = gfu.vec
#V[:,0] = np.array(V_sb[0])
V[:,0] = np.array(V_sb[0])/np.linalg.norm(np.array(V_sb[0]))
erg_h[:,0] = gfu.vec
S_POD[:,0] = gfu.vec
tend += 1
o = 1
e = int(line[o])
for i in range(int(dn)):
    conv.Assemble()
    conv_h[:,i] = conv.vec # stors the assembled convection vector in a matrix for the reducedbasis
    res.data = a.mat * gfu.vec + conv.vec
    gfu.vec.data -= dt * inv * res
    
    for r in line:
        if int(r) == i: #desided which solution will be orthogonalirsed
            if i > 0:
                with TaskManager():
                    V_sb.AppendOrthogonalize(gfu.vec)
                V[:,o] = np.array(V_sb[o])
                S_POD[:,o] = gfu.vec
                o = o + 1
    erg_h[:,i+1] = gfu.vec #stores the solution for comparing later
    t = t + dt
    Redraw()